In [1]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder

def preprocess_image(image_path, target_size=(128, 128)):  # Reduce target size
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0
    return image

# Example: Preprocess all images in a folder
def preprocess_dataset(data_dir, target_size=(224, 224)):
    images = []  # Initialize an empty list first
    labels = []
    class_names = os.listdir(data_dir)
    
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):  # Skip non-directory files
            continue
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = preprocess_image(image_path, target_size)
            images.append(image)
            labels.append(class_name)

    return np.array(images, dtype='float32'), np.array(labels)

# Preprocess the dataset
data_dir = "Data"
images, labels = preprocess_dataset(data_dir)


In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(labels)

# Convert numerical labels to one-hot encoding
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_labels = integer_labels.reshape(len(integer_labels), 1)
onehot_labels = onehot_encoder.fit_transform(integer_labels)

In [3]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, onehot_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

c:\Users\Arsh Bhushan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,172,314 (42.62 MB)

 Trainable params: 11,172,314 (42.62 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 140s 512ms/step - accuracy: 0.8088 - loss: 0.7444 - val_accuracy: 0.9948 - val_loss: 0.0215
Epoch 2/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 130s 490ms/step - accuracy: 0.9983 - loss: 0.0061 - val_accuracy: 0.9853 - val_loss: 0.0784
Epoch 3/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 133s 503ms/step - accuracy: 0.9941 - loss: 0.0339 - val_accuracy: 0.9981 - val_loss: 0.0030
Epoch 4/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 135s 508ms/step - accuracy: 0.9999 - loss: 2.8419e-04 - val_accuracy: 1.0000 - val_loss: 2.7351e-04
Epoch 5/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 129s 488ms/step - accuracy: 1.0000 - loss: 3.4624e-05 - val_accuracy: 0.9995 - val_loss: 9.0129e-04
Epoch 6/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 127s 480ms/step - accuracy: 1.0000 - loss: 3.8978e-06 - val_accuracy: 0.9995 - val_loss: 8.7775e-04
Epoch 7/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 130s 490ms/step - accuracy: 1.0000 - loss: 3.3178e-06 - val_accuracy: 0.9995 - val_loss: 8.8232e-04
Epoch 8/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 128s

In [6]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.9992 - loss: 0.0072
Test Accuracy: 99.85%


In [7]:
# Save the model
model.save("AtoZsign_language_model.h5")

In [9]:
print(model.input_shape)


(None, 224, 224, 3)


In [8]:
# Load the model
model = tf.keras.models.load_model("AtoZsign_language_model.h5")

# Preprocess a new image
new_image = preprocess_image("Data/C/single_24.jpg")

# Make a prediction
predictions = model.predict(np.array([new_image]))
predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]
print(f"Predicted Class: {predicted_class}")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 86528, but received input with shape (1, 25088)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 128, 128, 3), dtype=float32)
  • training=False
  • mask=None